## Installs Necessários
Para rodar a tela foram necessários os instal's =
- [pandas](https://pandas.pydata.org/docs/)
- [panel](https://panel.holoviz.org)
- [psycopg2](https://www.psycopg.org/docs/)
- [hvplot](https://hvplot.holoviz.org)
- [jupyter_bokeh](https://docs.bokeh.org/en/3.2.2/docs/user_guide/output/jupyter.html)
- [nbconvert](https://nbconvert.readthedocs.io/en/latest/)

## Importação / Conexão;
Aqui eu resolvi utilizar a **psycopg2** para manipular o banco de dados e além das que a professora pediu resolvi importar a hvplot que apenas ajuda a gerar alguns gráficos

In [113]:
import pandas as pd
import psycopg2 as pg
import panel as pn
import hvplot.pandas

BokehModel(combine_events=True, render_bundle={'docs_json': {'0e048e44-9dd9-457a-bd31-47d5e005a0b2': {'version…

In [114]:
#Conectando ao banco

conn = pg.connect(
    host = "localhost",
    database = "testp",
    user = "postgres",
    password ="jabuticaba12"
)

In [115]:
cursor = conn.cursor()

In [116]:
pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'2aef6471-fc4c-48df-8e84-766fa231eeba': {'version…

In [117]:
cursor.execute("select * from candidato")

## Criação DataFrames
Aqui eu construi algumas DataFrames, onde no método tabulator revolsi usar o layout um pouco diferente, com esse parâmetro buttons ele gera botões que conseguem ordenar a saida por ASC ou DESC

In [118]:
result = cursor.fetchall()
df = pd.DataFrame(result, columns=['id', 'nome', 'idade', 'email', 'telefone', 'escolaridade', 'experiencia', 'perfil_acessibilidade', 'tipo_deficiencia'])
df_widget = pn.widgets.Tabulator(df, buttons={'Print': "<i class='fa fa-print'></i>"})

## Criação de botões e Gráficos
Aqui estão reunidos todas declarações de botões e definições que fiz durante o código, assim como a construção de gráficos com auxílio do hvplot

### Botões/Informações/Funções da parte de consulta

In [119]:
flag=''

nome = pn.widgets.TextInput(name="Nome", placeholder='Digite o nome')
telefone = pn.widgets.TextInput(name="Telefone", placeholder='Digite o telefone')
email = pn.widgets.TextInput(name= "Email", placeholder= 'Digite o Email' )
buttonConsultar = pn.widgets.Button(name='Consultar', button_type='primary')
scroll_idade = pn.widgets.IntSlider(name = 'Idade menor/igual a', value= 30,  start = 18, end = 60, bar_color='#5353ec')
checkbox_group = pn.widgets.CheckBoxGroup(
    name="Grau de Escolaridade", 
    options=['Ensino superior completo', 'Ensino superior incompleto', 'Ensino médio completo', 
             'Ensino médio incompleto', 'Ensino técnico completo', 'Ensino técnico incompleto', 'Pós-graduação'],
    inline=True
)


#### Gráficos
Como citei, usei o hvplot para construir esses gráficos, o gráfico de linhas ficou um pouco bruto mas é por conta que o banco de dados ainda tem poucos dados.

In [120]:
# Widget do gráfico
def create_chart(dataframe):
    if dataframe.empty:
        return "Sem dados para exibir"
    return dataframe['escolaridade'].value_counts().hvplot.bar(
        title="Distribuição da Escolaridade",
        xlabel="Nível de Escolaridade",
        ylabel="Quantidade",
        rot=45,
        height=400,
        width=600,
        color="blue"
    )



grafico_escolaridade = pn.pane.HoloViews(create_chart(df), linked_axes=False)

In [121]:
def create_age_chart(df):
    if df.empty:
        return "Sem dados para exibir"
    return df['idade'].value_counts().sort_index().hvplot.line(
        title="Distribuição de Idades",
        xlabel="Idade",
        ylabel="Quantidade",
        height=400,
        width=600,
        line_width=2,
        color="red"
    )

grafico_idade = pn.pane.HoloViews(create_age_chart(df), linked_axes=False)

#### Definição do que o botão consultar faz
Eu decidi separar a parte da consulta da parte da edição pra conseguir ter mais nitidez na hora de mostrar as alterações na tela.

In [122]:
def on_consultar(event):
    try:
        conn.rollback()  # Corrige transações pendentes

        query = """
        SELECT * FROM candidato 
        WHERE (((%s = %s AND %s = %s AND %s = %s) OR nome = %s OR telefone = %s or email = %s) AND idade <= %s)
        """
        params = (nome.value, flag, telefone.value, flag, email.value, flag, nome.value, telefone.value, email.value, scroll_idade.value)
        if checkbox_group.value:
            escolaridades = "', '".join(checkbox_group.value)  # Converte lista para string formatada
            query += f" AND escolaridade IN ('{escolaridades}')"

        cursor.execute(query, tuple(params))

        result_c = cursor.fetchall()
        df_c = pd.DataFrame(result_c, columns=['id', 'nome', 'idade', 'email', 'telefone', 
                                               'escolaridade', 'experiencia', 'perfil_acessibilidade', 'tipo_deficiencia'])

        # Atualiza o widget da tabela
        df_widget.value = df_c

    except Exception as e:
        conn.rollback()
        pn.state.notifications.error(f'Erro ao consultar: {str(e)}')

### Botões/Informações/Funções da parte de edição

In [123]:
nome_i = pn.widgets.TextInput(name="Nome", placeholder='Digite o nome')
idade_i = pn.widgets.TextInput(name="Idade", placeholder='Digite a idade')
email_i = pn.widgets.TextInput(name="Email", placeholder='Digite o email')
telefone_i = pn.widgets.TextInput(name="Telefone", placeholder='Digite o telefone')
escolaridade_i = pn.widgets.RadioBoxGroup(name='Escolaridade',  options=['Ensino superior completo', 'Ensino superior incompleto', 'Ensino médio completo', 
             'Ensino médio incompleto', 'Ensino técnico completo', 'Ensino técnico incompleto', 'Pós-graduação'], inline=True)
experiencia_i = pn.widgets.TextInput(name="Experiencia", placeholder='Digite a experiencia')
perfil_acessibilidade_i = pn.widgets.TextInput(name="Perfil Acessibilidade", placeholder='digite o perfil')
tipo_deficiencia_i = pn.widgets.MultiChoice(name='Tipo Deficiência', options=['Deficiência Visual', 'Deficiência Física', 'Deficiência Auditiva', 'Deficiência Intelectual', 'Deficiência Psicossocial'])
id_i = pn.widgets.TextInput(name="ID (necessário somente se for atualizar/deletar)", placeholder='Digite o ID')

buttonInserir = pn.widgets.Button(name='Inserir/Atualizar', button_type='primary')
buttonDeletar = pn.widgets.Button(name='Deletar', button_type='primary')

#### Definição do que inserir faz
Aqui o mesmo botão de inserção também faz a atualização, isso é verificado pelo campo "id" se o usuário preencher esse campo ao inserir, o sistema vai entender que ele tá tendando atualizar um
candidato já existente e vai fazer a busca pelo id passado, caso exista, ele atualiza, e caso o usuário não tenha passado o campo id, ele cria um novo.

In [124]:
def on_inserir(event):
    try:
        conn.rollback()  # Garante que não há transações pendentes

        # 🔹 Validando campos obrigatórios
        if not nome_i.value or not idade_i.value or not email_i.value or not telefone_i.value or not escolaridade_i.value:
            pn.state.notifications.error("Preencha todos os campos obrigatórios!")
            return
        
        # 🔹 Validação de idade
        try:
            idade = int(idade_i.value)
        except ValueError:
            pn.state.notifications.error("A idade deve ser um número válido!")
            return
        
        tipo_deficiencia = ', '.join(tipo_deficiencia_i.value) if tipo_deficiencia_i.value else None

        if id_i.value:  # 🔹 Se o usuário informou um ID, tentar atualizar
            try:
                id_value = int(id_i.value)
                cursor.execute("SELECT 1 FROM candidato WHERE id = %s", (id_value,))
                if not cursor.fetchone():
                    pn.state.notifications.error(f"ID {id_value} não encontrado! Verifique e tente novamente.")
                    return
                
                update_query = """
                UPDATE candidato
                SET nome = %s, idade = %s, email = %s, telefone = %s, escolaridade = %s, 
                    experiencia = %s, perfil_acessibilidade = %s, tipo_deficiencia = %s
                WHERE id = %s
                """
                
                cursor.execute(update_query, (
                    nome_i.value, idade, email_i.value, telefone_i.value, escolaridade_i.value,
                    experiencia_i.value or None, perfil_acessibilidade_i.value or None, tipo_deficiencia, id_value
                ))

                conn.commit()
                pn.state.notifications.success("Candidato atualizado com sucesso!")
                
            
            except ValueError:
                pn.state.notifications.error("O ID deve ser um número válido!")
                return
        else:
            cursor.execute("SELECT MAX(id) FROM candidato")
            max_id = cursor.fetchone()[0]
            new_id = max_id + 1 if max_id is not None else 1

            insert_query = """
            INSERT INTO candidato (id, nome, idade, email, telefone, escolaridade, experiencia, perfil_acessibilidade, tipo_deficiencia)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(insert_query, (
                new_id, nome_i.value, idade, email_i.value, telefone_i.value, escolaridade_i.value,
                experiencia_i.value or None, perfil_acessibilidade_i.value or None, tipo_deficiencia
            ))

            conn.commit()
            pn.state.notifications.success(f"Candidato inserido com sucesso! ID gerado: {new_id}")

            
        cursor.execute("SELECT * FROM candidato")
        result = cursor.fetchall()
        df_update = pd.DataFrame(result, columns=['id', 'nome', 'idade', 'email', 'telefone', 
                                                  'escolaridade', 'experiencia', 'perfil_acessibilidade', 'tipo_deficiencia'])
        
        df_widget.value = df_update  # Atualiza tabela
        grafico_escolaridade.object = create_chart(df_update) # Atualiza o gráfico de escolaridade
        grafico_idade.object = create_age_chart(df_update)  # Atualiza gráfico de idade

        # 🔹 Limpando os campos de inserção após sucesso
        nome_i.value = ''
        idade_i.value = ''
        email_i.value = ''
        telefone_i.value = ''
        escolaridade_i.value = None
        experiencia_i.value = ''
        perfil_acessibilidade_i.value = ''
        tipo_deficiencia_i.value = []
        id_i.value = ''

    except Exception as e:
        conn.rollback()
        pn.state.notifications.error(f"Erro ao inserir: {str(e)}")

#### Definição do que Deletar faz
Deletar é mais simples que inserir, o usuário passa um id novamente e clicar em deletar, se o id existir no banco, o candidato será deletado. 

In [125]:
def on_deletar(event):
    try:
        conn.rollback()
        
        if not id_i.value:  # 🔹 Verifica se o ID foi informado
            pn.state.notifications.error("Informe um ID para deletar!")
            return
        
        try:
            id_value = int(id_i.value)
        except ValueError:
            pn.state.notifications.error("O ID deve ser um número válido!")
            return
        
        # 🔹 Verifica se o ID existe no banco
        cursor.execute("SELECT 1 FROM candidato WHERE id = %s", (id_value,))
        if not cursor.fetchone():
            pn.state.notifications.error(f"Nenhum candidato encontrado com o ID {id_value}.")
            return
        
        # 🔹 Deleta o candidato
        delete_query = "DELETE FROM candidato WHERE id = %s"
        cursor.execute(delete_query, (id_value,))
        conn.commit()
        
        pn.state.notifications.success(f"Candidato com ID {id_value} deletado com sucesso!")
        
        # 🔹 Atualizar tabela e gráficos após deleção
        cursor.execute("SELECT * FROM candidato")
        result = cursor.fetchall()
        df_update = pd.DataFrame(result, columns=['id', 'nome', 'idade', 'email', 'telefone', 
                                                  'escolaridade', 'experiencia', 'perfil_acessibilidade', 'tipo_deficiencia'])
        
        df_widget.value = df_update  # Atualiza tabela
        grafico_escolaridade.object = create_chart(df_update)  # Atualiza gráfico de escolaridade
        grafico_idade.object = create_age_chart(df_update)  # Atualiza gráfico de idade
        
        # 🔹 Limpa o campo de ID após a deleção
        id_i.value = ''

    except Exception as e:
        conn.rollback()
        pn.state.notifications.error(f"Erro ao deletar candidato: {str(e)}")


### Atribuindo as funções aos botões

In [126]:
buttonConsultar.on_click(on_consultar)
buttonInserir.on_click(on_inserir)
buttonDeletar.on_click(on_deletar)

Watcher(inst=Button(button_type='primary', name='Deletar'), cls=<class 'panel.widgets.button.Button'>, fn=<function on_deletar at 0x000001756DA268E0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

## Criação de Layout

In [127]:
layout = pn.Column(
    "# Consulta de Candidatos",
    pn.Row(  # Linha com os filtros de busca
        pn.Column(nome, telefone, email, scroll_idade, checkbox_group, buttonConsultar),
        pn.Spacer(width=50),
    ),
    pn.Column(df_widget),
    pn.Row(  # Linha com os gráficos lado a lado
        grafico_escolaridade,
        grafico_idade
    ),
    pn.layout.Divider(),  # Separador visual
    "# Edição de Candidatos",
    pn.Row(  # Organizando os inputs em colunas para melhor visualização
        pn.Column(
            nome_i,
            idade_i,
            email_i,
            telefone_i,
            escolaridade_i
        ),
        pn.Spacer(width=45),
        pn.Column(
            experiencia_i,
            perfil_acessibilidade_i,
            tipo_deficiencia_i,
            id_i,
            pn.Row( buttonInserir, buttonDeletar)  # Botão para inserir candidato
        )
    )
)


In [128]:
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'f6fb8ad0-a634-47e1-8589-ccc693e0854e': {'version…